# Decision trees, ensemble methods and Random Forests

Decision trees are very popular in machine learning. They contain a number of favourable properties, amongst which:
- Interpretability,
- Visualization,
- Able to handle missing values and 'mixed' data (e.g. a dataset consisting of a mix of categorical and continuous features),
- Able to deal with non-linearity,
- No need to normalize the data,
- Naturally extendable to a multiclass setting.

When many randomized decision trees are bagged in a single model, we obtain the powerful _Random Forest_ classifier (check out [this paper](http://jmlr.org/papers/volume15/delgado14a/delgado14a.pdf), where 179 classifiers from 17 families have been evaluated on 121 datasets). The Random Forest classifier is among the most popular machine learning methods available due to its high performance on most problems, not needing much or any tuning and the possibility of providing variable importance indices. The training of a random forest model can easily be done in parallel (e.g. fitting multiple trees on separate CPUs), making it a suitable method for large-scale data applications. Furthermore, it inherits the favourable properties of decision trees listed above, such that for example the method is easily extended for multi-output problems.

In this PC lab, we will use the [Breast Cancer Wisconsin (Diagnostic) Data Set](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic)). Run the following codeblock to download the dataset.

In [ ]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/BioML-UGent/MLLS/main/09_trees/wdbc.data", "wdbc.data")

## Decision trees

When a decision tree is fitted to the data, the feature space is recursively splitted. Decision trees make use of a purity measure in order to choose a feature to make a split. Such a purity measure assesses the homogeneity of the labels within the leaf of a decision tree. You have seen two such measures during the theory lecture, the entropy and gini-index measures. In a $K$-class classification setting, having class labels $1, ..., K$, the purity of a set of instances $\{(x_1,y_1), ..., (x_n,y_n)\}$ in a leaf can be defined using the proportions $p_1, ..., p_K$ (where $I$ is defined as the identity-function):

\begin{equation}
p_k = \frac{\sum_{i=1}^{n}I(y_i = k)}{n}, \qquad k = 1, ..., K,
\end{equation}

\begin{equation}
\text{entropy} = -\sum_{k=1}^{K}p_k \ln{p_k}, \qquad \text{gini-index} = \sum_{k=1}^{K} p_k(1-p_k).
\end{equation}


These purity measures can be used to split the feature space until a certain leaf is pure. Decision trees are called a _greedy_ approach, as they will choose that feature which will result in the greatest drop for a purity measure at a certain split, not taking the effect of future splits into account.  

To start of, let's investigate whether the decision tree can handle the XOR problem, a classic toy dataset for which a model needs to be able to handle non-linearities. For this we will use a helper function:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


''' Function to plot the decision boundary '''
''' X_train: Training dataset to visualize. Needs to be a numpy array of two dimensions.
The first two components are visualized '''
''' clf: Fitted classifier (also to two dimensions) '''


def plot_decision_boundary(X_train, y_train, clf):
    plot_step = 0.02
    x1_min, x1_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
    x2_min, x2_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
    x1, x2 = np.meshgrid(np.arange(x1_min, x1_max, plot_step),np.arange(x2_min, x2_max, plot_step))
    Z = clf.predict(np.c_[x1.ravel(), x2.ravel()])
    Z = Z.reshape(x1.shape)

    blues = y_train == 0
    reds = y_train == 1

    fig, ax = plt.subplots()
    ax.contourf(x1, x2, Z, cmap=plt.cm.Paired)
    ax.scatter(X_train[reds,0],X_train[reds,1], c='red');
    ax.scatter(X_train[blues,0],X_train[blues,1], c='blue');
    ax.set_xlabel(r'$x_1$', size=18)
    ax.set_ylabel(r'$x_2$', size=18)
    plt.show()

Generate the dataset:

In [ ]:
X_train = np.random.randn(400, 2)
y_train = np.logical_xor(X_train[:, 0] > 0, X_train[:, 1] > 0)

blues = y_train == 0
reds = y_train == 1

fig, ax = plt.subplots()
ax.scatter(X_train[reds,0],X_train[reds,1], c='red');
ax.scatter(X_train[blues,0],X_train[blues,1], c='blue');
ax.set_xlabel(r'$x_1$', size=18)
ax.set_ylabel(r'$x_2$', size=18)
plt.show()

We can fit decision trees using the already-familiar sklearn interface ([link to docs](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)).

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)

plot_decision_boundary(X_train, y_train, clf)

<div class="alert alert-success">
<b>EXERCISE (warm-up):
<br>
Think about some other classifiers we have already discussed during this course. How will they square up against the XOR problem? What kind of decision boundary do you expect in the case of e.g. logistic regression? Try plugging some classifiers into the code above and test if the results confirm your intuition.
</b>
</div>

## Pruning decision trees

A naive implementation of a decision tree would grow the number of splits the tree makes until it has correctly predicted all training labels, hence making for a model with high variance (potentially leading to overfitting). Just like other machine learning methods, decision trees can be regularized in various ways (using the [sklearn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)).

<div class="alert alert-success">
<b>EXERCISE (warm-up):
<br>
Look into the documentation and identify some hyperparameters that would regularize a decision tree. What would be your preferred way to regularize trees?
</b>
</div>

Let's code some stuff on a real dataset. For this we read in the previously-mentioned wisconsin breast cancer dataset:

In [ ]:
data = pd.read_csv(
    './wdbc.data',
    index_col=0)
status = data['status']
display(data.head(5))
print('Number of benign cases: ' + str(data[data.loc[:,'status']=='B'].shape[0]))
print('Total number of cases: ' + str(data.shape[0]))

<div class="alert alert-success">
<b>EXERCISE:
<br>
Construct a decision tree model that can predict whether a tumor is malign or benign.
Evaluate said model. Optionally, tune your model to make it as good as possible.
</b>
</div>

Some considerations to ponder on:
- Your dataset is quite small, how does this influence your data splitting (e.g. percentages) strategy?
- The labels are unbalanced, how does this influence (1) learning and (2) evaluation? For (2), let's say this data + model pipeline is a first preliminary test as a patient comes in, how could you account for this in your evaluation?

## Random Forests

Random Forests are a very popular method used in various fields, ranging from bioinformatics to data competitions (have a look at [this introduction](http://onlinelibrary.wiley.com/doi/10.1002/widm.1072/abstract)  for computational biologists and bioinformaticians). This is because they are known for their high performance, yet they are easy to implement. Random Forests make use of two kinds of randomization:
- The method aggregates the result of individual unpruned trees, grown on bootstrap (sub)samples of the dataset; this is called _bagging_ (**b**ootstrap **agg**regat**ing**).
- Whenever a decision tree makes a split, only a random subset of the present features are offered to the decision tree.

Random Forests are part of a larger class of models which are called _ensemble models_. The power of ensemble methods can be attributed to some kind of ["wisdom of the crowd"](https://en.wikipedia.org/wiki/Wisdom_of_the_crowd) phenomenon: averaging the results of many "simple" models is often better than relying on one "advanced" model, just as the collective judgement of a large group of "average" persons may be more reliable than that from one "expert". For random forests, the two kinds of randomization (+ optionally other kinds of regularization) aid in making the individual trees "simple".



After model fitting, for classification, a majority vote for all trees $T_m$ is taken to make a prediction for a new element $x_{new}$:
\begin{equation}
\hat{y}_{new} = 0 \quad \text{if} \quad  \sum_{m=1}^M T_m(x_{new}) / M < 0.5, \\
\hat{y}_{new} = 1 \quad \text{if} \quad  \sum_{m=1}^M T_m(x_{new}) / M > 0.5.   
\end{equation}

For regression problems, the average across trees is simply taken.


The [scikit-learn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) of random forests works similarly to that of decision trees, with some different parameters. The two most important hyperparameters are the number of trees that is grown and the number of subsetted features that is offered when making a split. The default values of these parameters usually lead to a good performance, although it is of course recommended to do some model selection.

<div class="alert alert-success">

<b>EXERCISE:
<br>
Repeat the previous model analysis. Use the same data split you made earlier. Investigate what features are important for prediction according to the model.
</b>
</div>

At the end of this exercise, you will (normally) have one final score for your decision tree model and one for your random forest model, allowing comparison between the two.

Let's say you worked for a hospital, and you wanted to report your results to the hospital department that would use your model and for instance say "I have a model that is x% accurate", would you use the scores obtained in this analysis?

Additionally, how would you explain the feature importances to a biologist looking at your results? How do you think the feature importances are influenced by the feature resampling hyperparameter in the model?

Note: If you implement tuning here, take note that for every model hyperparameter setup you are evaluating, you are in fact evaluating an ensemble of smaller models. I.e. Random forests are inherently more computationally demanding than single trees. What kind of effect does this has on your hyperparameter tuning, what can you do?

## Extra: XGBoost

Boosting is a subclass of ensembling in which (similarly) many simpler models team up to create a performant model. The difference is that in boosting, subsequent trees try to correct the mistakes of the previous one(s). The most famous example, XGBoost or eXtreme Gradient Boosting is a decision-tree-based algorithm that uses a gradient boosting framework. The algorithm was developed by Tianqi Chen and Carlos Guestrin and more details can be found [here](https://arxiv.org/pdf/1603.02754.pdf). The technique has managed to win numerous kaggle competitions and is currently tied to an active open source [project](https://github.com/dmlc/xgboost/).

**Informally, performant boosting implementations such as XGBoost, [CatBoost](https://catboost.ai/) and [LightGBM](https://lightgbm.readthedocs.io/en/v3.3.2/), have been crowned as kings of machine learning on tabular, structured datasets. (As opposed to various deep learning architectures which have been informally crowned as kings of "perceptual", unstructured data such as, language, vision, audio, ...)**



Some of the main characteristics that make XGboost an appealing choice include:
- Can be used to solve different type of prediction problems (regression, classification, ranking)
- Runs on every popular OS (Windows, Linux, and OS X)
- Supports all major programming languages (C++, Python, R, Java, Scala, and Julia)


The library we are going to use is quite flexible as it supports the following cases:

- Trees are constructed in parallel using all of available CPU cores during training.
- Very large models can use a cluster of machines in a distributed fashion.
- Very large datasets that don’t fit into memory are handled with out-of-Core Computing.

There is no additional exercise linked to XGBoost. It suffices to say that the XGBoost python package can be installed with the cell below and that the package uses a similar API as in scikit learn: `.fit()`, `.predict()`, ... More information on input arguments here: [XGBRegressor](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBRegressor) and [XGBClassifier](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBClassifier). We invite you to play around with the model by plugging XGBoost in the above exercise.

In [ ]:
!pip install xgboost